In [0]:
import tensorflow as tf
import meteo_path as me

In [0]:
import altair as alt
import pdvega

In [0]:
z = tf.Variable(1.0, name='z')
eps = tf.Variable(1.0, name='eps')

In [0]:
a = tf.Variable(1.0, name='a')
b = tf.Variable(1.0, name='b')

In [0]:
eqr = me.rename(a**2-1+z*(7-5*a)-b**2, 'real')
eqi = me.rename(b*(2*a-5*z)-eps, 'imag')

In [0]:
mod = me.Model([z, eps], [a, b], [eqr, eqi])

In [0]:
tf.Gra

In [0]:
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    mod.solve_system(sess=sess, output=True)
    ret = mod.eval_system(sess=sess)
me.summary(ret)

In [0]:
par0 = {z: 0.0, eps: 0.001}
par1 = {z: 0.1, eps: 0.001}
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    mod.set_params(par0, sess=sess)
    mod.solve_system(sess=sess, output=True)
    print(mod.eval_system(sess=sess))
    t_path, par_path, var_path = mod.homotopy_path(par1, output=True, sess=sess)

In [0]:
pd.DataFrame(var_path, columns=['a', 'b']).vgplot()